In [291]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, GridSearchCV


# data loading

In [260]:
user_df = pd.read_csv(r"data_for_db\users.csv")
user_review_df = pd.read_csv(r"data_for_db\user_reviews.csv")
user_wishlist_df = pd.read_csv(r"data_for_db\user_wishlists.csv")
book_df = pd.read_csv(r"data_for_db\books.csv")
author_df = pd.read_csv(r"data_for_db\authors.csv")
publisher_df = pd.read_csv(r"data_for_db\publishers.csv")

C:\Users\Jimmy\AppData\Local\Temp\ipykernel_8104\72079949.py:1: DtypeWarning: Columns (0: username, 1: email) have mixed types. Specify dtype option on import or set low_memory=False.
  user_df = pd.read_csv(r"data_for_db\users.csv")


# data preparation

## remove deactive users & book

In [261]:
threshold_user=3
threshold_book=5

user_count = user_review_df.value_counts("user_id")
active_user_mask = user_count[user_count > threshold_user].index

book_count = user_review_df.value_counts("book_id")
active_book_mask = book_count[book_count > threshold_book].index

# filterd
user_review_df = user_review_df[user_review_df["user_id"].isin(active_user_mask) & user_review_df["book_id"].isin(active_book_mask)]

user_review_df.shape

(139656, 5)

In [262]:
len(user_review_df["user_id"].unique())

14580

In [263]:
len(user_review_df["book_id"].unique())

10725

user_review

In [264]:
user_review_df.sample(3)

,user_id,book_id,book_rating,comment,is_favourite
140923,98391,82774,9.0,NaN,False
156563,106849,82318,10.0,This is a comment,False
316884,228455,133278,7.0,This is a comment,False


In [265]:
user_review_df.value_counts("book_rating")

book_rating
8.0     34412
10.0    27467
9.0     24657
7.0     23521
5.0     13613
6.0     10688
4.0      2441
3.0      1656
2.0       758
1.0       443
Name: count, dtype: int64

In [243]:
# user_id_old = user_review_df["user_id"].unique()
# user_id_new = (range(len(user_id_old)))
# user_id_map_old2new = {id_old:str(id_new) for id_old,id_new in zip(user_id_old, user_id_new)}
# user_id_map_new2old = {id_new:str(id_old) for id_old,id_new in zip(user_id_old, user_id_new)}

# book_id_old = user_review_df["book_id"].unique()
# book_id_new = (range(len(user_id_old)))
# book_id_map_old2new = {id_old:str(id_new) for id_old,id_new in zip(book_id_old, book_id_new)}
# book_id_map_new2old = {id_new:str(id_old) for id_old,id_new in zip(book_id_old, book_id_new)}

In [244]:
# user_review_df["user_id_new"] = user_review_df["user_id"].map(user_id_map_old2new)
# user_review_df["book_id_new"] = user_review_df["book_id"].map(book_id_map_old2new)

In [245]:
user_review_df.sample(5)

,user_id,book_id,book_rating,comment,is_favourite
491860,134694,108149,NaN,This is a comment,False
790172,215821,170747,9.0,NaN,False
621375,171078,143147,NaN,This is a comment,False
575824,157811,236651,10.0,This is a comment,False
637303,174596,33689,8.0,This is a comment,False


In [280]:
user_review_df_plus = user_review_df.copy()
fav_plus = 3
user_review_df_plus.loc[user_review_df_plus["is_favourite"] == True, "book_rating"] += fav_plus

In [284]:
user_review_df_plus.value_counts("book_rating")

book_rating
8.0     34412
7.0     23521
10.0    21935
9.0     19708
5.0     13613
6.0     10688
13.0     5532
12.0     4949
4.0      2441
3.0      1656
2.0       758
1.0       443
Name: count, dtype: int64

In [297]:
# 2. Load data into Surprise format
# The Reader needs to know your rating scale (e.g., 1 to 5)
reader = Reader(rating_scale=(1, 13))
data = Dataset.load_from_df(user_review_df[['user_id', 'book_id', 'book_rating']], reader)

# 4. Initialize the SVD model
param_grid = {
    "n_factors": [50, 100, 150],
    "lr_all": [0.002, 0.005, 0.01],
    "reg_all": [0.02, 0.1, 0.4],
    "n_epochs": [20, 30]
}

# 2. Initialize GridSearchCV
# cv=3 means 3-fold cross-validation (it splits data 3 times to ensure stability)
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3, n_jobs=-1)

# 5. Train the model
gs.fit(data)

# 4. Results
print(f"Best RMSE score: {gs.best_score['rmse']:.4f}")
print(f"Best Parameters: {gs.best_params['rmse']}")



Best RMSE score: 1.5865
Best Parameters: {'n_factors': 50, 'lr_all': 0.01, 'reg_all': 0.4, 'n_epochs': 20}


In [298]:
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

RMSE: 1.1392
MAE:  0.8375


In [300]:
test_user_id = 	129084

test_book_id = 109023
pred = model.predict(test_user_id, test_book_id)    

print(f"Predicted rating for User {test_user_id} on Item {test_book_id}: {pred.est:.2f}")

Predicted rating for User 129084 on Item 109023: 7.65
